In [1]:
print("krunal")

krunal


In [2]:
%pwd

'd:\\Runway\\Projects\\Gen_AI\\rag_medical_bot\\research'

In [3]:
import os 
os.chdir("../")

In [4]:
%pwd

'd:\\Runway\\Projects\\Gen_AI\\rag_medical_bot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
#extracted_data

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [11]:
#from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

In [12]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


#its output come 384 becasue , the output of "sentence-transformers/all-MiniLM-L6-v2"is 384

In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [24]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [27]:
from langchain_pinecone import PineconeVectorStore

In [ ]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [28]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [29]:
docsearch

In [30]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [54]:
retrieved_docs = retriever.invoke("London?")

In [55]:
retrieved_docs

[Document(id='ef8299c3-93a1-456f-b033-52f71acff65a', metadata={'page': 414.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Medicine which he founded in the early 1980s. The center\nis located at 152 E. 55th St., New York, NY 10022.\n(AP/Wide World Photos. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:43 PM  Page 401'),
 Document(id='3480b43f-d60d-49da-a2cc-e08a35e3e95e', metadata={'page': 400.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Aston formed her own company called the Aston Para-\ndigm Corporation in Lake Tahoe, California. This com-\npany provides training and certification for Aston practi-\ntioners. She also began exploring how environmental\nconditions affect body movement, foreshadowing the\nergonomic movement in the workplace that developed\nin the 1990s. Over time, Aston has expanded her move-\nment work to include a fitness program for older adults.\nToday, Judith Aston serves as director of Aston Paradigm\nCorporation.'),
 Document(id='2

In [41]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [42]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [43]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is acne"})
print(response["answer"])



Acne is a common skin disease that causes pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. It is the most common skin disease, affecting nearly 17 million people in the United States.


In [53]:
response = rag_chain.invoke({"input": "how is the London ?"})
print(response["answer"])



I am an AI assistant and do not have personal experiences or knowledge of specific locations. However, I can provide information based on the context provided. The text mentions that London was a hub for various communities such as the artistic, medical, and scientific communities during the first half of the twentieth century. It was also a place where the Alexander Technique gained support from influential figures such as John Dewey, Aldous Huxley, and Bernard Shaw. Today, researchers continue to study the technique's effects and applications in fields such as education and performance.
